У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn.multiclass import OneVsRestClassifier

In [3]:
# Завантажуємо датасет

raw_df = pd.read_csv('data/customer_segmentation_train.csv')

In [4]:
# Превіряємо загальний вигляд даних

raw_df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [5]:
# Перевіряємо розмір вибірки

raw_df.shape

(8068, 11)

In [6]:
# Перевіряємо наявність і кількість порожніх значень у стовпцях

raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [7]:
# Рахуємо кількість і відсоток пропущених значень у кожному стовпці

null_count = raw_df.isnull().sum()
null_percentage = round((raw_df.isnull().sum()/raw_df.shape[0])*100, 2)

null_df = pd.DataFrame({'column_name' : raw_df.columns,'null_count' : null_count,'null_percentage': null_percentage})
null_df.reset_index(drop = True, inplace = True)

null_df.sort_values(by = 'null_percentage', ascending = False)

,column_name,null_count,null_percentage
6,Work_Experience,829,10.28
8,Family_Size,335,4.15
2,Ever_Married,140,1.74
5,Profession,124,1.54
4,Graduated,78,0.97
9,Var_1,76,0.94
0,ID,0,0.00
1,Gender,0,0.00
3,Age,0,0.00
7,Spending_Score,0,0.00


Кількість пропущених значень знаходиться в межах 10% для окремо взятих стовпців (в одному випадку трохи перевищує 10%). Тому жоден з них не видаляємо. Можемо застосувати імпутацію.

In [8]:
# Видаляємо стовпці, які зайві для моделі
raw_df.drop(['ID'], axis = 1, inplace = True)

# Розбиваємо вибірку на тренувальний і тестовий набори 
train_df, test_df = train_test_split(raw_df, test_size = 0.2, random_state = 42, stratify = raw_df.Segmentation)

input_cols = list(train_df.columns)[:-1]
target_col = list(train_df.columns)[-1]

train_inputs = train_df[input_cols].copy()
train_targets = train_df[target_col].copy()

test_inputs = test_df[input_cols].copy()
test_targets = test_df[target_col].copy()

# Виявляємо числові і категоріальні колонки
numeric_cols = train_inputs.select_dtypes(include = np.number).columns.tolist()
categorical_cols = train_inputs.select_dtypes(include = 'object').columns.tolist()

In [9]:
# Проводимо імпутацію для числових стовпців

imputer_num = SimpleImputer(strategy = 'mean')
imputer_num.fit(train_inputs[numeric_cols])
train_inputs[numeric_cols] = imputer_num.transform(train_inputs[numeric_cols])
test_inputs[numeric_cols] = imputer_num.transform(test_inputs[numeric_cols])

In [10]:
# Проводимо імпутацію для категоріальних стовпців

imputer_cat = SimpleImputer(strategy = 'most_frequent')
imputer_cat.fit(train_inputs[categorical_cols])
train_inputs[categorical_cols] = imputer_cat.transform(train_inputs[categorical_cols])
test_inputs[categorical_cols] = imputer_cat.transform(test_inputs[categorical_cols])

In [11]:
# Кодуємо категоріальні стовпці

onehot = OneHotEncoder(sparse = False, handle_unknown = 'ignore')
onehot.fit(train_inputs[categorical_cols])
encoded_cols = list(onehot.get_feature_names_out(categorical_cols))
train_inputs[encoded_cols] = onehot.transform(train_inputs[categorical_cols])
test_inputs[encoded_cols] = onehot.transform(test_inputs[categorical_cols])

In [12]:
# Створюємо остаточні набори, що містять лише ті стовпці, які підуть на вхід моделі

X_train = train_inputs[numeric_cols + encoded_cols]
X_test = test_inputs[numeric_cols + encoded_cols]

y_train = train_df[target_col]
y_test = test_df[target_col]

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [13]:
# SMOTENC oversampling

categorical_indexes = [X_train.columns.tolist().index(col) for col in encoded_cols]
smotenc = SMOTENC(random_state = 42, categorical_features = categorical_indexes)
X_train_sm, y_train_sm = smotenc.fit_resample(X_train, y_train)

In [14]:
# SMOTETomek resampling

smotetomec = SMOTETomek(random_state = 42, smote = smotenc)
X_train_smt, y_train_smt = smotetomec.fit_resample(X_train, y_train)

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [15]:
# Тренування моделі логістичної регресії з використанням стратегії One-vs-Rest на даних, збалансованих з Smotenc

log_reg_sm = LogisticRegression(solver = 'liblinear')
ovr_smotenc = OneVsRestClassifier(log_reg_sm)
ovr_smotenc.fit(X_train_sm, y_train_sm)
ovr_smotenc_predictions = ovr_smotenc.predict(X_test)

# Обчислення метрик для кожного класу

print(classification_report(y_test, ovr_smotenc_predictions))

              precision    recall  f1-score   support

           A       0.41      0.47      0.44       394
           B       0.40      0.23      0.29       372
           C       0.51      0.59      0.55       394
           D       0.67      0.73      0.70       454

    accuracy                           0.52      1614
   macro avg       0.50      0.50      0.49      1614
weighted avg       0.50      0.52      0.50      1614



In [16]:
# Тренування моделі логістичної регресії з використанням стратегії One-vs-Rest на даних, збалансованих з Smotetomek

log_reg_smt = LogisticRegression(solver = 'liblinear')
ovr_smotetomek = OneVsRestClassifier(log_reg_smt)
ovr_smotetomek.fit(X_train_smt, y_train_smt)
ovr_smotetomek_predictions = ovr_smotetomek.predict(X_test)

# Обчислення метрик для кожного класу

print(classification_report(y_test, ovr_smotetomek_predictions))

              precision    recall  f1-score   support

           A       0.43      0.47      0.45       394
           B       0.39      0.27      0.32       372
           C       0.52      0.57      0.54       394
           D       0.67      0.72      0.70       454

    accuracy                           0.52      1614
   macro avg       0.50      0.51      0.50      1614
weighted avg       0.51      0.52      0.51      1614



Для порівняння моделей з усіх виведених метрик краще дивитись на macro avg f1-score.\
Ця метрика практично однакова для двох випадків (0,49 і 0,50).\
Ситуація може пояснюватись тим, що у нашому наборі даних немає великої кількості зв’язків Томека.